In [6]:
%load_ext autoreload
%autoreload 2

from hydroml.models.get_model_from_path import get_model_from_path
from hydroml.config.config import load_config
from pathlib import Path
from hydroml.models import get_model

from hydroml.models.lstm import HydroLSTM

path='P://work//sho108//hydroml//results//default//241209185033_f34a'
model = HydroLSTM.load_from_checkpoint(Path(path)/'last.ckpt') #, config=config


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
model.config.dynamic_features
model.config.static_features
seqlen = model.config.seqlen



In [33]:
import numpy as np
import torch
from hydroml.utils import helpers as h
import pandas as pd
import xarray as xr
from hydroml.data.transformers import get_transformer

def f(x, s, model, date=None):
    
    if model.config.add_sin_cos_doy:
        dates = xr.DataArray(pd.date_range(start=date, periods=seqlen, freq='D'), dims=['time'])
        x_dynamic = np.concatenate([x, h.get_sin_cos_doy(dates)], axis=1) # should be come from helper function 

    # load transformer
    parameters = h.read_yaml(model.config.get_transform_parameters_path())
    parameters = parameters[model.config.target_features[0]]
    transformer = get_transformer(model.config.transform_target)(parameters=parameters)

    # transform
    x_dynamic = transformer.transform_dynamic(x_dynamic)
    

    y =  model.forward(torch.tensor(x_dynamic.astype(np.float32)), torch.tensor(s.astype(np.float32)))

    # inverse transform
    y = transformer.inverse_transform_dynamic(y)

    return y



date = '2024-01-01'
x_dynamic = np.random.rand(seqlen, len(model.config.dynamic_features))
x_static = np.random.rand(len(model.config.static_features)+len(model.config.evolving_static_features)).astype(np.float32)






In [34]:
f(x_dynamic, x_static, model, '2024-01-01'), f(x_dynamic, x_static, model, '2024-10-02')

ValueError: Unsupported transformer: {'model': 'HydroLSTM', 'lstm_dynamic_input_feature_latent_size': 1, 'lstm_static_input_latent_size': 1, 'lstm_hidden_size': 16, 'lstm_target_size': 1, 'lstm_layers': 1, 'lr': 0.001, 'dropout_probability': 0.0, 'initial_forget_bias': 1.0, 'weight_decay': 0.0, 'loss_fn': 'nse', 'number_of_time_output_timestep': 1, 'seqlen': 60, 'transform_predictor_dynamic': 'norm', 'transform_target': 'norm', 'dynamic_features': ['precipitation_AWAP', 'et_morton_wet_SILO'], 'target_features': ['streamflow_mmd'], 'static_features': ['catchment_area', 'mean_slope_pct', 'prop_forested', 'upsdist', 'strdensity', 'strahler', 'frac_snow', 'p_seasonality', 'p_mean', 'pet_mean', 'aridity', 'high_prec_freq', 'high_prec_dur'], 'evolving_static_features': {'dynamic_feature_mean': {'dynamic_features': ['precipitation_AWAP']}}, 'add_sin_cos_doy': True, 'batch_size': 128, 'dataloader_nworkers': 0, 'dataloader_persistent_workers': False, 'cal': {'periods': [['1985-01-01', '2017-01-